<a href="https://colab.research.google.com/github/RichardsRodrigues/Cotacoes_Cambiais/blob/main/cotacoes_cambiais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Pipeline de Cotações Cambiais com Python + LLM - Google Colab
# MBA Data Engineering - Projeto Final
# Autor: Richard Rodrigues Silva
# ====================================================================
# INSTALAÇÃO DE DEPENDÊNCIAS
# ====================================================================
# Instala bibliotecas necessárias
!pip install -q requests pandas pyarrow google-genai python-dotenv plotly

# ====================================================================
# IMPORTS NECESSÁRIOS
# ====================================================================
import os
import json
import logging
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Optional, Literal
import warnings
from dotenv import load_dotenv, set_key

# Importação correta para Google GenAI
try:
    from google import genai
    from google.genai.errors import APIError as GenAI_APIError
except ImportError:
    # Se não encontrar, define como None e usa o fallback
    genai = None
    GenAI_APIError = Exception

# Silencia warnings do pandas e outros para um output mais limpo no Colab
warnings.filterwarnings('ignore')

# ====================================================================
# CONFIGURAÇÃO DE LOGGING
# ====================================================================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# ====================================================================
# GESTÃO DE CHAVES E AMBIENTE
# ====================================================================

# Definindo tipo para maior clareza e controle
APIKeyName = Literal["EXCHANGE_API_KEY", "GEMINI_API_KEY"]
ENV_PATH = Path("/content/.env")

# Definição das chaves padrão (Mude para chaves reais ou deixe vazio/default)
DEFAULT_KEYS = {
    "EXCHANGE_API_KEY": "fc96490f94b914303644490e", # Chave de exemplo
    "GEMINI_API_KEY": ""                            # Deixe vazio ou adicione sua chave
}

def create_or_load_env():
    """Cria o .env se não existir, ou carrega as chaves existentes."""
    print(" CONFIGURAÇÃO DE AMBIENTE")
    print("=" * 50)

    if not ENV_PATH.exists():
        logger.info(" Arquivo .env não encontrado. Criando um novo.")
        env_content = []
        for key, value in DEFAULT_KEYS.items():
            final_value = value if value else f"SUA_CHAVE_{key}"
            env_content.append(f"{key}={final_value}")

        try:
            with open(ENV_PATH, 'w', encoding='utf-8') as f:
                f.write('\n'.join(env_content))
            logger.info(f" Arquivo .env criado em: {ENV_PATH}")
        except Exception as e:
            logger.error(f" Erro ao criar arquivo .env: {e}")

    # Garante que o .env é carregado (se existir)
    load_dotenv(ENV_PATH)

    # Carrega chaves após o load_dotenv
    return {
        key: os.getenv(key, "") for key in DEFAULT_KEYS.keys()
    }

def update_env_key(key_name: APIKeyName, new_value: str) -> bool:
    """Atualiza uma chave específica no arquivo .env e no ambiente."""
    try:
        if not ENV_PATH.exists():
            create_or_load_env() # Cria se o arquivo sumiu

        set_key(ENV_PATH, key_name, new_value)
        os.environ[key_name] = new_value
        logger.info(f" Chave {key_name} atualizada com sucesso.")
        show_env_status()
        return True
    except Exception as e:
        logger.error(f" Erro ao atualizar {key_name}: {e}")
        return False

def show_env_status():
    """Mostra o status atual das configurações, mascarando as chaves."""
    print("\n STATUS ATUAL DAS CHAVES DE API:")
    print("-" * 40)

    api_keys = create_or_load_env() # Garante o carregamento

    for key, value in api_keys.items():
        is_configured = value and value != f"SUA_CHAVE_{key}" and value != DEFAULT_KEYS[key]
        status = "✅ Configurada" if is_configured else "❌ Não configurada"

        if is_configured:
            masked_value = value[:6] + "*" * (len(value) - 10) + value[-4:] if len(value) > 12 else "***"
            print(f"{key}: {status} ({masked_value})")
        else:
            print(f"{key}: {status}")

    print("-" * 40)

# Inicialização e carregamento das chaves
api_keys = create_or_load_env()
EXCHANGE_API_KEY = api_keys["EXCHANGE_API_KEY"]
GEMINI_API_KEY = api_keys["GEMINI_API_KEY"]

show_env_status()

# ====================================================================
# CLASSE PRINCIPAL DO PIPELINE
# ====================================================================

class CurrencyPipelineColab:
    """
    Pipeline de Extração, Transformação, Carga (ETC) de cotações cambiais
    com enriquecimento via LLM do Google (Gemini).
    """

    def __init__(self, base_currency: str = "USD",
                 target_currencies: List[str] = None):

        self.base_currency = base_currency
        self.target_currencies = target_currencies or ["BRL", "EUR", "GBP", "JPY", "AUD", "CAD", "CHF", "CNY"]
        self.setup_directories()
        self.gemini_client = None

        if GEMINI_API_KEY and genai:
            try:
                # O parâmetro os.environ.get('GEMINI_API_KEY') é redundante se já usamos load_dotenv
                self.gemini_client = genai.Client(api_key=GEMINI_API_KEY)
                logger.info(" Gemini Client inicializado com sucesso.")
            except Exception as e:
                logger.error(f" Falha ao inicializar Gemini Client: {e}")
                self.gemini_client = None

    def setup_directories(self):
        """Cria estrutura de diretórios (Raw, Silver, Gold)."""
        self.data_dir = Path("/content/currency_data")
        self.raw_dir = self.data_dir / "raw"
        self.silver_dir = self.data_dir / "silver"
        self.gold_dir = self.data_dir / "gold"

        for directory in [self.raw_dir, self.silver_dir, self.gold_dir]:
            directory.mkdir(parents=True, exist_ok=True)

        logger.info(" Estrutura de diretórios configurada.")

    # --- ETAPA 1: INGESTÃO (EXTRAÇÃO) ---

    def ingest_exchange_rates(self, date: Optional[datetime] = None) -> Dict:
        """Coleta dados da API ExchangeRate e salva na camada RAW."""
        date = date or datetime.now()
        date_str = date.strftime("%Y-%m-%d")
        logger.info(f" Iniciando ingestão para {date_str} (Base: {self.base_currency})")

        if not EXCHANGE_API_KEY or EXCHANGE_API_KEY == "SUA_CHAVE_EXCHANGE_API" or EXCHANGE_API_KEY == DEFAULT_KEYS['EXCHANGE_API_KEY']:
             raise ValueError("API Key da ExchangeRate não configurada ou é a chave de exemplo.")

        try:
            url = f"https://v6.exchangerate-api.com/v6/{EXCHANGE_API_KEY}/latest/{self.base_currency}"

            response = requests.get(url, timeout=15)
            response.raise_for_status() # Lança erro para status 4xx/5xx

            data = response.json()

            if data.get("result") != "success":
                raise Exception(f"Erro da API: {data.get('error-type', 'Erro desconhecido')}")

            # Adiciona metadados
            data["ingestion_timestamp"] = datetime.now().isoformat()
            data["pipeline_version"] = "2.1.0"

            # Salva dados brutos
            raw_file = self.raw_dir / f"{date_str}.json"
            with open(raw_file, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2, ensure_ascii=False)

            logger.info(f" Dados brutos salvos com {len(data['conversion_rates'])} moedas.")
            return data

        except requests.exceptions.RequestException as e:
            logger.error(f" Erro de rede/API na ingestão: {e}")
            raise
        except Exception as e:
            logger.error(f" Erro na ingestão: {e}")
            raise

    # --- ETAPA 2: TRANSFORMAÇÃO ---

    def transform_data(self, raw_data: Dict, date: datetime) -> pd.DataFrame:
        """Normaliza, filtra e valida dados, salvando na camada SILVER."""
        logger.info(" Iniciando transformação dos dados.")

        try:
            rates = raw_data["conversion_rates"]

            # Normalização e filtro em uma única passagem
            records = [
                {
                    "date": date.strftime("%Y-%m-%d"),
                    "base_currency": raw_data["base_code"],
                    "target_currency": currency,
                    "exchange_rate": rate,
                    "last_update": raw_data["time_last_update_utc"],
                    "ingestion_timestamp": raw_data["ingestion_timestamp"]
                }
                for currency, rate in rates.items()
                if currency in self.target_currencies
            ]

            df = pd.DataFrame(records)

            self._validate_data_quality(df)

            # Salva dados transformados (CSV)
            date_str = date.strftime("%Y-%m-%d")
            silver_file = self.silver_dir / f"{date_str}.csv"
            df.to_csv(silver_file, index=False)

            logger.info(f" Transformação concluída: {len(df)} registros processados na camada Silver.")
            return df

        except Exception as e:
            logger.error(f" Erro na transformação: {e}")
            raise

    def _validate_data_quality(self, df: pd.DataFrame):
        """Validações de qualidade dos dados (interna)."""
        logger.info(" Executando validações de qualidade.")

        # Verifica taxas inválidas (<= 0)
        invalid_rates = df[df["exchange_rate"] <= 0]
        if not invalid_rates.empty:
            logger.warning(f" ❌ Qualidade: {len(invalid_rates)} taxas inválidas encontradas.")

        # Verifica moedas ausentes
        missing_currencies = set(self.target_currencies) - set(df["target_currency"].unique())
        if missing_currencies:
            logger.warning(f" ⚠️ Qualidade: Moedas alvo ausentes: {missing_currencies}")

    # --- ETAPA 3: CARGA ---

    def load_to_gold(self, df: pd.DataFrame, date: datetime) -> str:
        """Salva dados otimizados em formato Parquet na camada GOLD."""
        logger.info(" Carregando para camada GOLD (Parquet).")

        try:
            date_str = date.strftime("%Y-%m-%d")
            gold_file = self.gold_dir / f"{date_str}.parquet"

            # Otimizações (tipagem e ordenação)
            df["date"] = pd.to_datetime(df["date"])
            df = df.sort_values(["date", "target_currency"])

            # Salva em Parquet (otimizado para leitura)
            df.to_parquet(gold_file, index=False, compression="snappy")

            logger.info(f" Dados salvos em Parquet: {gold_file}")
            return str(gold_file)

        except Exception as e:
            logger.error(f" Erro na carga para GOLD: {e}")
            raise

    # --- ETAPA 4: ENRIQUECIMENTO LLM ---

    def generate_llm_insights(self, df: pd.DataFrame, date: datetime) -> str:
        """Gera insights usando a API do Google (Gemini) ou um fallback básico."""
        logger.info(" Iniciando geração de insights (LLM).")

        if self.gemini_client:
            try:
                # Prepara dados e constrói o prompt
                analysis_data = self._prepare_analysis_data(df)
                prompt = self._build_analysis_prompt(analysis_data, date)

                # Chamada ao modelo
                response = self.gemini_client.models.generate_content(
                    model="gemini-2.5-flash",
                    contents=prompt
                )
                logger.info(" Insights gerados com sucesso pelo Gemini.")
                return response.text
            except GenAI_APIError as e:
                logger.warning(f" ⚠️ Falha na API do Gemini: {e}. Gerando insights básicos.")
                return self._generate_basic_insights(df, date)
            except Exception as e:
                logger.warning(f" ⚠️ Erro inesperado no LLM: {e}. Gerando insights básicos.")
                return self._generate_basic_insights(df, date)
        else:
            logger.info(" API Gemini não configurada ou importação falhou. Gerando insights básicos.")
            return self._generate_basic_insights(df, date)

    def _prepare_analysis_data(self, df: pd.DataFrame) -> Dict:
        """Prepara e agrega dados-chave para o prompt do LLM."""
        return {
            "total_currencies": len(df),
            "strongest_currency": df.loc[df["exchange_rate"].idxmin(), "target_currency"],
            "weakest_currency": df.loc[df["exchange_rate"].idxmax(), "target_currency"],
            "average_rate": df["exchange_rate"].mean(),
            # Filtros mais robustos para moedas específicas
            "brl_rate": df.loc[df["target_currency"] == "BRL", "exchange_rate"].iloc[0] if "BRL" in df["target_currency"].values else "N/A",
            "eur_rate": df.loc[df["target_currency"] == "EUR", "exchange_rate"].iloc[0] if "EUR" in df["target_currency"].values else "N/A",
            "top_5_rates": df.nlargest(5, "exchange_rate")[["target_currency", "exchange_rate"]].to_dict("records")
        }

    def _build_analysis_prompt(self, data: Dict, date: datetime) -> str:
        """Constrói o prompt detalhado para o LLM."""
        top_5_rates_str = "\n".join([f"- {item['target_currency']}: {item['exchange_rate']:.4f}" for item in data['top_5_rates']])

        return f"""
        Você é um analista financeiro sênior. Analise as cotações cambiais de {date.strftime('%d/%m/%Y')} (base USD):

        DADOS PRINCIPAIS:
        - Total de moedas analisadas: {data['total_currencies']}
        - Moeda mais forte (menor taxa USD/Moeda): {data['strongest_currency']}
        - Moeda mais fraca (maior taxa USD/Moeda): {data['weakest_currency']}
        - Taxa média geral: {data['average_rate']:.4f}

        DESTAQUES PARA O BRASIL E EURO:
        - USD/BRL: {data['brl_rate']:.4f}
        - USD/EUR: {data['eur_rate']:.4f}

        TOP 5 MAIORES TAXAS (As moedas que mais desvalorizaram frente ao USD):
        {top_5_rates_str}

        SOLICITAÇÃO:
        Forneça uma análise executiva em português com a seguinte estrutura e estilo profissional:

        1. **RESUMO EXECUTIVO** (3-4 linhas sobre o cenário geral e a posição do USD)
        2. **ANÁLISE DO REAL (BRL)** (Comente sobre o valor do USD/BRL e suas implicações)
        3. **MOVIMENTOS CHAVE** (Destaque as 2 moedas que mais se destacaram [forte e fraca] e o porquê - se possível, inferir o impacto global)
        4. **RECOMENDAÇÕES TÁTICAS** (Ações práticas para gestores financeiros brasileiros)

        Mantenha linguagem clara, objetiva e voltada para tomadores de decisão. Não inclua texto introdutório além do seu título.
        """

    def _generate_basic_insights(self, df: pd.DataFrame, date: datetime) -> str:
        """Função de fallback (insights básicos) caso o LLM falhe ou não esteja configurado."""
        strongest = df.loc[df["exchange_rate"].idxmin()]
        weakest = df.loc[df["exchange_rate"].idxmax()]
        brl_rate = df.loc[df["target_currency"] == "BRL", "exchange_rate"].iloc[0] if "BRL" in df["target_currency"].values else "N/A"

        return f"""
ANÁLISE BÁSICA - {date.strftime('%d/%m/%Y')}

==================================================
RESUMO EXECUTIVO:
- {len(df)} moedas analisadas em relação ao USD.
- Moeda mais forte hoje: {strongest['target_currency']} ({strongest['exchange_rate']:.4f})
- Moeda mais fraca hoje: {weakest['target_currency']} ({weakest['exchange_rate']:.4f})

DESTAQUE BRASIL:
- USD/BRL: {brl_rate:.4f}

Esta é uma análise básica (fallback). Para insights de mercado e recomendações detalhadas, configure sua chave GEMINI_API_KEY no arquivo .env.
==================================================
"""

    # --- ETAPA 5: VISUALIZAÇÕES (MELHORADA) ---

    def create_visualizations(self, df: pd.DataFrame):
        """
        Cria e exibe visualizações dos dados, focando em clareza e interatividade.
        """
        logger.info(" Criando visualizações interativas.")

        # 1. Preparação dos dados para visualização
        df_sorted = df.copy().sort_values("exchange_rate")

        # Adiciona coluna de Rank (posição da moeda frente ao USD)
        df_sorted['Rank'] = df_sorted['exchange_rate'].rank(method='min').astype(int)

        # Adiciona texto informativo para hover
        df_sorted['hover_text'] = (
            'Rank: ' + df_sorted['Rank'].astype(str) + '<br>' +
            'Taxa: ' + df_sorted['exchange_rate'].round(4).astype(str) +
            f'<br>(1 {self.base_currency} = X)'
        )

        # 2. Gráfico de Barras Interativo (Plotly)
        print(" 📈 Gerando Gráfico de Cotações Interativo...")

        fig1 = px.bar(
            df_sorted,
            x="target_currency",
            y="exchange_rate",
            title=f"Cotações Cambiais em {df_sorted['date'].iloc[0]} (Base: {self.base_currency})",
            color="exchange_rate",
            color_continuous_scale="RdYlGn_r", # Escala de cor invertida (menor taxa = verde)
            text=df_sorted['exchange_rate'].round(4),
            hover_data={'target_currency': False, 'exchange_rate': False, 'hover_text': True}
        )

        fig1.update_layout(
            xaxis_title="Moeda Alvo",
            yaxis_title=f"Taxa de Câmbio (Valor da Moeda em {self.base_currency})",
            title_x=0.5,
            height=500,
            coloraxis_colorbar=dict(title="Taxa")
        )

        fig1.update_traces(
            texttemplate='%{text}',
            textposition='outside',
            hovertemplate='<b>Moeda: %{x}</b><br>%{customdata[0]}<extra></extra>' # Usa o hover_text
        )

        fig1.show()

        # 3. Tabela de Resumo Detalhado
        print("\n 📊 RESUMO ESTATÍSTICO DETALHADO:")
        print("-" * 50)

        summary_df = df_sorted[['target_currency', 'exchange_rate', 'Rank']].rename(
            columns={'exchange_rate': 'Taxa (USD=X)', 'target_currency': 'Moeda'}
        ).set_index('Moeda')

        # Usa to_markdown para formatação amigável no Colab
        print(summary_df.to_markdown(floatfmt=".4f", numalign="center", stralign="center"))
        print("-" * 50)
        logger.info(" Visualizações concluídas.")
        return fig1

    # --- EXECUÇÃO PRINCIPAL ---

    def run_full_pipeline(self, date: Optional[datetime] = None) -> Dict:
        """Executa todas as etapas do pipeline em sequência."""
        date = date or datetime.now()

        print("\n" + "="*65)
        print("    PIPELINE DE COTAÇÕES CAMBIAIS - INICIANDO EXECUÇÃO")
        print("="*65)
        print(f" Data de Referência: {date.strftime('%d/%m/%Y')} | Moeda Base: {self.base_currency}")
        print(f" Moedas Alvo: {', '.join(self.target_currencies)}")
        print("="*65)

        try:
            # 1. Ingestão
            print("\n 🚀 ETAPA 1: INGESTÃO (RAW)")
            raw_data = self.ingest_exchange_rates(date)

            # 2. Transformação
            print("\n ✨ ETAPA 2: TRANSFORMAÇÃO (SILVER)")
            transformed_df = self.transform_data(raw_data, date)

            # 3. Carga
            print("\n 💾 ETAPA 3: CARGA (GOLD)")
            gold_file = self.load_to_gold(transformed_df, date)

            # 4. Insights LLM
            print("\n 🤖 ETAPA 4: ANÁLISE INTELIGENTE")
            insights = self.generate_llm_insights(transformed_df, date)

            # 5. Visualizações
            print("\n 📈 ETAPA 5: VISUALIZAÇÕES")
            self.create_visualizations(transformed_df)

            # Exibe insights
            print("\n" + "="*65)
            print(" INSIGHTS GERADOS PELA INTELIGÊNCIA ARTIFICIAL:")
            print("="*65)
            print(insights)
            print("="*65)

            result = {
                "status": "SUCESSO",
                "date": date.strftime("%Y-%m-%d"),
                "records_processed": len(transformed_df),
                "gold_file": gold_file,
                "llm_used": bool(self.gemini_client),
                "currencies_analyzed": self.target_currencies
            }

            logger.info(" Pipeline concluído com sucesso.")
            print(f"\n✅ PIPELINE CONCLUÍDO COM SUCESSO! {len(transformed_df)} registros processados.")
            return result

        except Exception as e:
            logger.critical(f" Falha Crítica no pipeline: {e}")
            print(f"\n 🛑 ERRO CRÍTICO NO PIPELINE: {e}")
            return {
                "status": "ERRO",
                "error": str(e),
                "date": date.strftime("%Y-%m-%d")
            }

# ====================================================================
# FUNÇÕES UTILITÁRIAS PARA O NOTEBOOK
# ====================================================================

def update_exchange_key(new_key: str):
    """Atualiza a chave da ExchangeRate API e recarrega o ambiente."""
    update_env_key("EXCHANGE_API_KEY", new_key)

def update_gemini_key(new_key: str):
    """Atualiza a chave da Gemini API e recarrega o ambiente."""
    update_env_key("GEMINI_API_KEY", new_key)

# ====================================================================
# EXECUÇÃO PRINCIPAL
# ====================================================================

def main():
    """Função principal - Execute esta célula para rodar o pipeline"""
    global EXCHANGE_API_KEY, GEMINI_API_KEY

    # Recarrega as chaves antes da execução, caso tenham sido alteradas
    current_keys = create_or_load_env()
    EXCHANGE_API_KEY = current_keys["EXCHANGE_API_KEY"]
    GEMINI_API_KEY = current_keys["GEMINI_API_KEY"]

    show_env_status()

    # Validação final da chave de cotações (obrigatória)
    if not EXCHANGE_API_KEY or EXCHANGE_API_KEY == "SUA_CHAVE_EXCHANGE_API" or EXCHANGE_API_KEY == DEFAULT_KEYS['EXCHANGE_API_KEY']:
        print("\n ⚠️ ALERTA CRÍTICO: Chave da ExchangeRate API não configurada ou é a chave de exemplo!")
        print(" Use: update_exchange_key('sua_chave_aqui') para configurar.")
        print(" Chave obrigatória para rodar o pipeline.")
        return

    try:
        # Inicializa e executa pipeline
        pipeline = CurrencyPipelineColab()
        result = pipeline.run_full_pipeline()

        # Mostra resultado final
        print(f"\n LOG DE EXECUÇÃO:")
        print("-" * 50)
        for key, value in result.items():
            print(f"   {key}: {value}")
        print("-" * 50)

    except Exception as e:
        logger.critical(f"Erro na execução principal: {e}")
        print(f" ERRO FATAL: Falha ao iniciar ou executar o pipeline. Verifique o log.")

# ====================================================================
# INSTRUÇÕES DE USO (Final)
# ====================================================================

print(f"""

  PIPELINE DE COTAÇÕES CAMBIAIS v2.1 (FINAL)
================================================================

  INSTRUÇÕES DE USO:

1.  COLOQUE AS CHAVES:
    • Se necessário, use as funções abaixo para atualizar as chaves:
    • **update_exchange_key('SUA_CHAVE')**
    • **update_gemini_key('SUA_CHAVE')**

2.  EXECUÇÃO:
    • O pipeline está pronto. Execute **main()** para rodar!

3.  FLUXO DE DADOS:
    • Raw (JSON) -> Silver (CSV) -> Gold (Parquet).

================================================================

  EXECUTE: main() para iniciar o pipeline!

""")

 CONFIGURAÇÃO DE AMBIENTE

 STATUS ATUAL DAS CHAVES DE API:
----------------------------------------
 CONFIGURAÇÃO DE AMBIENTE
EXCHANGE_API_KEY: ❌ Não configurada
GEMINI_API_KEY: ❌ Não configurada
----------------------------------------


  PIPELINE DE COTAÇÕES CAMBIAIS v2.1 (FINAL)

  INSTRUÇÕES DE USO:

1.  COLOQUE AS CHAVES:
    • Se necessário, use as funções abaixo para atualizar as chaves:
    • **update_exchange_key('SUA_CHAVE')**
    • **update_gemini_key('SUA_CHAVE')**

2.  EXECUÇÃO:
    • O pipeline está pronto. Execute **main()** para rodar!

3.  FLUXO DE DADOS:
    • Raw (JSON) -> Silver (CSV) -> Gold (Parquet).


  EXECUTE: main() para iniciar o pipeline!




In [ ]:
main()

 CONFIGURAÇÃO DE AMBIENTE

 STATUS ATUAL DAS CHAVES DE API:
----------------------------------------
 CONFIGURAÇÃO DE AMBIENTE
EXCHANGE_API_KEY: ✅ Configurada (129e35**************d44e)
GEMINI_API_KEY: ✅ Configurada (AIzaSy*****************************8bMY)
----------------------------------------

    PIPELINE DE COTAÇÕES CAMBIAIS - INICIANDO EXECUÇÃO
 Data de Referência: 01/10/2025 | Moeda Base: USD
 Moedas Alvo: BRL, EUR, GBP, JPY, AUD, CAD, CHF, CNY

 🚀 ETAPA 1: INGESTÃO (RAW)

 ✨ ETAPA 2: TRANSFORMAÇÃO (SILVER)

 💾 ETAPA 3: CARGA (GOLD)

 🤖 ETAPA 4: ANÁLISE INTELIGENTE

 📈 ETAPA 5: VISUALIZAÇÕES
 📈 Gerando Gráfico de Cotações Interativo...



 📊 RESUMO ESTATÍSTICO DETALHADO:
--------------------------------------------------
|  Moeda  |  Taxa (USD=X)  |  Rank  |
|:-------:|:--------------:|:------:|
|   GBP   |     0.7437     | 1.0000 |
|   CHF   |     0.7967     | 2.0000 |
|   EUR   |     0.8520     | 3.0000 |
|   CAD   |     1.3916     | 4.0000 |
|   AUD   |     1.5132     | 5.0000 |
|   BRL   |     5.3206     | 6.0000 |
|   CNY   |     7.1254     | 7.0000 |
|   JPY   |    147.9930    | 8.0000 |
--------------------------------------------------

 INSIGHTS GERADOS PELA INTELIGÊNCIA ARTIFICIAL:
**Análise Executiva de Cotações Cambiais - 01/10/2025**

**1. RESUMO EXECUTIVO**
Em 01/10/2025, o cenário cambial global aponta para uma valorização robusta do Dólar Americano (USD) frente à maioria das moedas analisadas, conforme a desvalorização significativa de moedas asiáticas e de commodities. Embora o GBP se destaque como exceção, a taxa média geral elevada, puxada pelo JPY, reflete a força do USD em um ambiente de volatilida